In [1]:
import yeelight as yl
from yeelight import discover_bulbs
import time
from pynput import mouse, keyboard

In [9]:
keyboard.Key.__dict__

mappingproxy({'_generate_next_value_': <function enum.Enum._generate_next_value_(name, start, count, last_values)>,
              '__module__': 'pynput.keyboard._win32',
              '__doc__': 'An enumeration.',
              '_member_names_': ['alt',
               'alt_l',
               'alt_r',
               'alt_gr',
               'backspace',
               'caps_lock',
               'cmd',
               'cmd_r',
               'ctrl',
               'ctrl_l',
               'ctrl_r',
               'delete',
               'down',
               'end',
               'enter',
               'esc',
               'f1',
               'f2',
               'f3',
               'f4',
               'f5',
               'f6',
               'f7',
               'f8',
               'f9',
               'f10',
               'f11',
               'f12',
               'f13',
               'f14',
               'f15',
               'f16',
               'f17',
               'f

In [14]:
yl.PowerMode.NORMAL

<PowerMode.NORMAL: 1>

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
from LightGroup import LightGroup

In [9]:
lg = LightGroup('bedroom', ['bedroom1'])

In [10]:
lg.bulbs

In [11]:
target = lg.degrees
def on_scroll(x, y, dx, dy):
    global target
#     print('Scrolled {0} at {1}'.format(
#         'down' if dy < 0 else 'up',
#         (x, y)))
    
    if dy < 0:
        target -= 50
        
    elif dy > 0:
        target += 50
        
    target = min(6700, max(target, 1700))  
    lg.set_color_temp(target)

lg.toggle_music_mode(1)
# Collect events until released
if globals().get('listener'):
    listener.stop()
listener = mouse.Listener(
        on_scroll=on_scroll)
listener.start()

In [13]:
listener.stop()
lg.toggle_music_mode('0')

'0'

In [14]:
target = lg.brightness
def on_scroll(x, y, dx, dy):
    global target
#     print('Scrolled {0} at {1}'.format(
#         'down' if dy < 0 else 'up',
#         (x, y)))
    
    if dy < 0:
        target -= 5
        
    elif dy > 0:
        target += 5
        
    target = min(100, max(target, 0))  
    lg.set_brightness(target)

lg.toggle_music_mode(1)
# Collect events until released
if globals().get('listener'):
    listener.stop()
listener = mouse.Listener(
        on_scroll=on_scroll)
listener.start()

In [15]:
listener.stop()
lg.toggle_music_mode('0')

'0'

In [10]:
lg.save_state('bedroom.txt')

In [12]:
lg = LightGroup('bedroom', ['bedroom1'], 'bedroom.txt')

{'on': 'True', 'mode': '1', 'h': '0', 's': '0', 'brightness': '10', 'degrees': '2600', 'r': '255', 'g': '255', 'b': '255', 'group_name': 'bedroom', 'names': 'bedroom1'}


In [23]:
discover_bulbs()

[{'ip': '192.168.1.230',
  'port': 55443,
  'capabilities': {'id': '0x00000000158acbe2',
   'model': 'color4',
   'fw_ver': '34',
   'support': 'get_prop set_default set_power toggle set_bright set_scene cron_add cron_get cron_del start_cf stop_cf set_ct_abx adjust_ct set_name set_adjust adjust_bright adjust_color set_rgb set_hsv set_music udp_sess_new udp_sess_keep_alive udp_chroma_sess_new',
   'power': 'on',
   'bright': '100',
   'color_mode': '2',
   'ct': '4650',
   'rgb': '16744474',
   'hue': '27',
   'sat': '89',
   'name': 'bedroom1'}}]

In [36]:
bulbs = get_bulbs(['bedroom1'])
bulbs[0].get_properties()['music_on']

'0'

In [38]:
prev_t = time.time()
def set_color_temp_slider(temp):
    global prev_t
    
    if time.time() < prev_t + .1:
        return
    set_color_temp(bulbs, temp)
    prev_t = time.time()

toggle_music_mode(bulbs, state=1)
interact(set_color_temp_slider, temp=widgets.IntSlider(min=1500, max=6700, step=50, value=4000))

interactive(children=(IntSlider(value=4000, description='temp', max=6700, min=1500, step=50), Output()), _dom_…

<function __main__.set_color_temp_slider(temp)>

In [31]:
bulbs[0].bulb.__dict__

{'_ip': '192.168.1.230',
 '_port': 55443,
 'effect': 'smooth',
 'duration': 300,
 'auto_on': False,
 'power_mode': <PowerMode.LAST: 0>,
 '_model': None,
 '_Bulb__cmd_id': 112,
 '_last_properties': {'power': 'on',
  'bright': '100',
  'ct': '3750',
  'rgb': '2544895',
  'hue': '192',
  'sat': '85',
  'color_mode': '2',
  'flowing': '0',
  'delayoff': '0',
  'music_on': '0',
  'name': 'bedroom1',
  'bg_power': None,
  'bg_flowing': None,
  'bg_ct': None,
  'bg_bright': None,
  'bg_hue': None,
  'bg_sat': None,
  'bg_rgb': None,
  'nl_br': None,
  'active_mode': None,
  'current_brightness': '100'},
 '_capabilities': {'id': '0x00000000158acbe2',
  'model': 'color4',
  'fw_ver': '34',
  'support': 'get_prop set_default set_power toggle set_bright set_scene cron_add cron_get cron_del start_cf stop_cf set_ct_abx adjust_ct set_name set_adjust adjust_bright adjust_color set_rgb set_hsv set_music udp_sess_new udp_sess_keep_alive udp_chroma_sess_new',
  'power': 'on',
  'bright': '100',
  'color